<a href="https://colab.research.google.com/github/KeishiIrisa/scraping/blob/main/%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0%E6%9C%AC%E7%95%AA%E7%92%B0%E5%A2%83_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
from bs4 import BeautifulSoup
import time
import re
import csv
new_url_list = []

with open('url_list_4_22.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        new_url_list.append(row)


def get_elements(soup, class_name):
    return soup.find_all("div", {"class": class_name})

def get_company_info(soup, title):
    for div in soup.find_all('div', {'class': 'rn3-companyOfferCompany__info'}):
        h3 = div.find('h3', {'class': 'rn3-companyOfferCompany__heading'})
        if h3 and h3.text.strip() == title:
            p = div.find('p', {'class': 'rn3-companyOfferCompany__text'})
            if p:
                return p.text.strip()
    return ""

def get_contact_info(soup, title):
    for div in soup.find_all('div', {'class': 'rn3-companyOfferEntry__info'}):
        h3 = div.find('h3', {'class': 'rn3-companyOfferEntry__heading'})
        if h3 and h3.text.strip() == title:
            text_div = div.find('div', {'class': 'rn3-companyOfferEntry__text'})
            if text_div:
                contact_link = text_div.find('a')['href']
                new_contact_link = "https://next.rikunabi.com" + contact_link
                return new_contact_link
    return ""



results = []

for url in new_url_list:
    try:
        # URLからページのHTMLを取得する
        response = requests.get(url)
        response.encoding = 'shift_jis'  # エンコーディングを指定
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # ページから要素を取得する
        company_name = get_elements(soup, "rn3-companyOfferRecruitment__headText")[0].text.strip()
        job_info = get_elements(soup, "rn3-companyOfferRecruitment__text")[0].text.strip()
        recruit_info = get_elements(soup, "rn3-companyOfferRecruitment__text")[1].text.strip()
        acquire_info = get_elements(soup, "rn3-companyOfferAcquire__headText")[0].text.strip()
        
        # 会社情報を取得する
        company = get_company_info(soup, "社名")
        establish = get_company_info(soup, "設立")
        representative = get_company_info(soup, "代表者")
        capital = get_company_info(soup, "資本金")
        sales = get_company_info(soup, "売上高")
        employees = get_company_info(soup, "従業員数")
        office = get_company_info(soup, "事業所")
        industry = get_company_info(soup, "業種")
        business = get_company_info(soup, "事業内容")
        
        # 連絡先情報を取得する
        contact_url = get_contact_info(soup, "連絡先")
        
        entry_info = get_elements(soup, "rn3-companyOfferEntry__text")[2].text.strip()
        
        # 要素を辞書に追加する
        results.append({
            "company_name": company_name if company_name else "",
            "job_info": job_info if job_info else "", 
            "recruit_info": recruit_info if recruit_info else "",
            "acquire_info": acquire_info if acquire_info else "",
            "company": company if company else "",
            "establish": establish if establish else "",
            "representative": representative if representative else "",
            "capital": capital if capital else "",
            "sales": sales if sales else "",
            "employees": employees if employees else "",
            "office": office if office else "",
            "industry": industry if industry else "",
            "business": business if business else "",
            "contact_url": contact_url if contact_url else "",
            "entry_info": entry_info
        })

        # 1秒間のウェイトを追加する
        time.sleep(1)
        
    except Exception as e:
        print(f"Error scraping {url}: {e}")


for result in results:
    for key in result.keys():
        value = result[key]
        if type(value) == str:  # 値が文字列の場合
            result[key] = value.replace('\n', '').replace('\r', '').replace('\xa0', '').replace('\u3000', '')
        elif type(value) == list:  # 値がリストの場合
            result[key] = [s.replace('\n', '').replace('\r', '').replace('\xa0', '').replace('\u3000', '') for s in value]


with open("new_results.csv", mode="w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["company_name", "job_info", "recruit_info", "acquire_info", "entry_info", "company", "contact_url" , "establish", "representative", "capital", "sales", "employees", "office", "industry", "business"])
    writer.writeheader()
    for result in results:
        row = {
            "company_name": result.get("company_name", ""),
            "job_info": result.get("job_info", ""),
            "recruit_info": result.get("recruit_info", ""),
            "acquire_info": result.get("acquire_info", ""),
            "entry_info": result.get("entry_info", ""),
            "company": result.get("company", ""),
            "contact_url": result.get("contact_url", ""),
            "establish": result.get("establish", ""),
            "representative": result.get("representative", ""),
            "capital": result.get("capital", ""),
            "sales": result.get("sales", ""),
            "employees": result.get("employees", ""),
            "office": result.get("office", ""),
            "industry": result.get("industry", ""),
            "business": result.get("business", "")
        }
        writer.writerow(row)

Error scraping ['https://next.rikunabi.com/company/cmi1960401002/nx2_rq0024727997/?fr=cp_s00700&list_disp_no=1&leadtc=n_ichiran_cst_n5_ttl']: No connection adapters were found for "['https://next.rikunabi.com/company/cmi1960401002/nx2_rq0024727997/?fr=cp_s00700&list_disp_no=1&leadtc=n_ichiran_cst_n5_ttl']"
Error scraping ['https://next.rikunabi.com/company/cmi1950118004/nx2_rq0023274125/?fr=cp_s00700&list_disp_no=2&leadtc=n_ichiran_cst_n5_ttl']: No connection adapters were found for "['https://next.rikunabi.com/company/cmi1950118004/nx2_rq0023274125/?fr=cp_s00700&list_disp_no=2&leadtc=n_ichiran_cst_n5_ttl']"
Error scraping ['https://next.rikunabi.com/company/cmi0502939471/nx2_rq0024490218/?fr=cp_s00700&list_disp_no=3&leadtc=n_ichiran_cst_n5_ttl']: No connection adapters were found for "['https://next.rikunabi.com/company/cmi0502939471/nx2_rq0024490218/?fr=cp_s00700&list_disp_no=3&leadtc=n_ichiran_cst_n5_ttl']"
Error scraping ['https://next.rikunabi.com/company/cmi0543756003/nx2_rq00246

In [ ]:
results

[]